# Plotting Examples with Matplotlib and Plotly

This notebook demonstrates how to visualize broadbean pulse sequences using both **matplotlib** and **plotly** backends.

## Table of Contents

* [Setup](#Setup)
* [Creating Sample Pulses](#Creating-Sample-Pulses)
* [Matplotlib Backend](#Matplotlib-Backend)
  * [Plotting Blueprints](#Plotting-Blueprints-with-Matplotlib)
  * [Plotting Elements](#Plotting-Elements-with-Matplotlib)
  * [Plotting Sequences](#Plotting-Sequences-with-Matplotlib)
* [Plotly Backend](#Plotly-Backend)
  * [Plotting Blueprints](#Plotting-Blueprints-with-Plotly)
  * [Plotting Elements](#Plotting-Elements-with-Plotly)
  * [Plotting Sequences](#Plotting-Sequences-with-Plotly)
* [Comparing Backends](#Comparing-Backends)

## Setup

First, let's import the necessary modules.

In [ ]:
import matplotlib as mpl

import broadbean as bb
from broadbean.plotting import plotter

# Configure matplotlib for better display
mpl.rcParams["figure.figsize"] = (10, 4)
mpl.rcParams["figure.subplot.bottom"] = 0.15

## Creating Sample Pulses

Let's create some sample blueprints, elements, and sequences that we'll use for plotting demonstrations.

In [ ]:
# Get the built-in pulse atoms
ramp = bb.PulseAtoms.ramp  # args: start, stop
sine = bb.PulseAtoms.sine  # args: freq, ampl, off, phase

# Create a simple blueprint with multiple segments
bp = bb.BluePrint()
bp.insertSegment(0, ramp, (0, 0.5e-3), name="ramp_up", dur=2e-6)
bp.insertSegment(1, sine, (2e6, 0.5e-3, 0.5e-3, 0), name="oscillation", dur=3e-6)
bp.insertSegment(2, ramp, (0.5e-3, 0), name="ramp_down", dur=2e-6)
bp.setSR(1e9)  # 1 GS/s sample rate

# Add some markers for demonstration
bp.setSegmentMarker("ramp_up", (0, 1e-6), 1)  # Marker 1 during first half of ramp_up
bp.setSegmentMarker("oscillation", (0.5e-6, 2e-6), 2)  # Marker 2 during oscillation

print("Blueprint created:")
bp.showPrint()

In [ ]:
# Create a second blueprint for multi-channel demonstration
bp2 = bb.BluePrint()
bp2.insertSegment(0, ramp, (0, -0.3e-3), name="init", dur=2e-6)
bp2.insertSegment(1, ramp, (-0.3e-3, -0.3e-3), name="hold", dur=3e-6)
bp2.insertSegment(2, ramp, (-0.3e-3, 0), name="release", dur=2e-6)
bp2.setSR(1e9)

print("Second blueprint created:")
bp2.showPrint()

In [ ]:
# Create an element with two channels
elem = bb.Element()
elem.addBluePrint(1, bp)
elem.addBluePrint(2, bp2)

print(f"Element created with {len(elem._data)} channels")
print(f"Total duration: {bp.duration * 1e6:.1f} µs")

In [ ]:
# Create a sequence with multiple positions
seq = bb.Sequence()
seq.setSR(1e9)

# Add the element at position 1
seq.addElement(1, elem)

# Create a variation of the element for position 2
bp3 = bp.copy()
bp3.changeArg("oscillation", "freq", 4e6)  # Double the frequency

elem2 = bb.Element()
elem2.addBluePrint(1, bp3)
elem2.addBluePrint(2, bp2)

seq.addElement(2, elem2)

# Set AWG specs
seq.setChannelAmplitude(1, 2.5)  # 2.5 V amplitude
seq.setChannelAmplitude(2, 2.5)
seq.setChannelOffset(1, 0)
seq.setChannelOffset(2, 0)

print(f"Sequence created with {seq.length_sequenceelements} positions")

## Matplotlib Backend

The matplotlib backend is the default plotting backend. It creates static plots that are great for documentation and publications.

### Plotting Blueprints with Matplotlib

In [ ]:
# Plot a blueprint using matplotlib (default backend)
fig = plotter(bp)

# The plotter returns the matplotlib figure, which can be customized further
fig.suptitle("Blueprint with Matplotlib", fontsize=12, y=1.02)

In [ ]:
# Explicitly specify the matplotlib backend
fig = plotter(bp, backend="matplotlib")
fig.suptitle("Explicit Matplotlib Backend", fontsize=12, y=1.02)

### Plotting Elements with Matplotlib

Elements show all channels side by side.

In [ ]:
# Plot the multi-channel element
fig = plotter(elem, backend="matplotlib")
fig.suptitle("Two-Channel Element with Matplotlib", fontsize=12, y=1.02)

### Plotting Sequences with Matplotlib

Sequences show all positions and channels in a grid layout.

In [ ]:
# Plot the full sequence
fig = plotter(seq, backend="matplotlib")
fig.suptitle("Sequence with Matplotlib (2 positions × 2 channels)", fontsize=12, y=1.02)

## Plotly Backend

The plotly backend creates interactive plots that allow zooming, panning, and hovering over data points.

**Note:** To use the plotly backend, you need to have plotly installed:
```bash
pip install plotly
```
or
```bash
pip install broadbean[plotly]
```

### Plotting Blueprints with Plotly

In [ ]:
# Plot a blueprint using the plotly backend
fig = plotter(bp, backend="plotly")

# Update the layout title
fig.update_layout(title="Blueprint with Plotly - Interactive!")
fig.show()

### Plotting Elements with Plotly

In [ ]:
# Plot the multi-channel element with plotly
fig = plotter(elem, backend="plotly")
fig.update_layout(
    title="Two-Channel Element with Plotly",
    height=500,  # Adjust height for better visibility
)
fig.show()

### Plotting Sequences with Plotly

In [ ]:
# Plot the full sequence with plotly
fig = plotter(seq, backend="plotly")
fig.update_layout(title="Sequence with Plotly (2 positions × 2 channels)", height=600)
fig.show()

## Comparing Backends

Here's a quick summary of when to use each backend:

| Feature | Matplotlib | Plotly |
|---------|------------|--------|
| Interactive zoom/pan | ❌ | ✅ |
| Hover information | ❌ | ✅ |
| Static export (PNG, PDF) | ✅ Excellent | ✅ Good |
| Publication quality | ✅ | ✅ |
| Jupyter notebook | ✅ | ✅ |
| No extra dependencies | ✅ | ❌ (requires plotly) |

**Recommendations:**
- Use **matplotlib** for publications, reports, and when you need fine control over the figure styling.
- Use **plotly** for interactive exploration, debugging pulse shapes, and presentations.

In [ ]:
# Create a more complex blueprint to showcase the backends
bp_complex = bb.BluePrint()
bp_complex.insertSegment(0, ramp, (0, 1e-3), name="charge", dur=1e-6)
bp_complex.insertSegment(1, sine, (5e6, 0.2e-3, 1e-3, 0), name="pulse", dur=2e-6)
bp_complex.insertSegment(2, ramp, (1e-3, 0.5e-3), name="measure", dur=1e-6)
bp_complex.insertSegment(3, ramp, (0.5e-3, 0.5e-3), name="hold", dur=2e-6)
bp_complex.insertSegment(4, ramp, (0.5e-3, 0), name="reset", dur=1e-6)
bp_complex.setSR(1e9)

# Add markers
bp_complex.setSegmentMarker("pulse", (0, 2e-6), 1)
bp_complex.setSegmentMarker("measure", (0, 1e-6), 2)

print("Complex blueprint structure:")
bp_complex.showPrint()

In [ ]:
# Matplotlib version
print("Matplotlib output:")
fig_mpl = plotter(bp_complex, backend="matplotlib")
fig_mpl.suptitle("Complex Pulse - Matplotlib", y=1.02)

In [ ]:
# Plotly version
print("Plotly output (interactive - try zooming!):")
fig_plotly = plotter(bp_complex, backend="plotly")
fig_plotly.update_layout(title="Complex Pulse - Plotly (Interactive)")
fig_plotly.show()